In [1]:
import tensorflow as tf
print('TensorFlow Version: {}'.format(tf.__version__))
print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.7.1
Default GPU Device: 


##### >**Note:** Make sure you have OpenAI Gym cloned. Then run this command `pip install -e gym/[all]`.

In [2]:
import numpy as np
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')

In [3]:
def model_input(state_size):
    #states = tf.placeholder(tf.float32, [None, *state_size], name='states')
    states = tf.placeholder(tf.float32, [None, state_size], name='states')
    actions = tf.placeholder(tf.int32, [None], name='actions')
    next_states = tf.placeholder(tf.float32, [None, state_size], name='next_states')
    rewards = tf.placeholder(tf.float32, [None], name='rewards')
    dones = tf.placeholder(tf.float32, [None], name='dones')
    rates = tf.placeholder(tf.float32, [None], name='rates') # success rate
    return states, actions, next_states, rewards, dones, rates

In [4]:
def Act(states, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('Act', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=hidden_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        h2 = tf.layers.dense(inputs=nl1, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
        
        # Output layer
        logits = tf.layers.dense(inputs=nl2, units=action_size)        
        return logits

In [5]:
def Env(states, actions, state_size, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('Env', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=action_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        nl1_fused = tf.concat(axis=1, values=[nl1, actions])
        h2 = tf.layers.dense(inputs=nl1_fused, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
                
        # Output layer
        states_logits = tf.layers.dense(inputs=nl2, units=state_size, trainable=False)
        Qlogits = tf.layers.dense(inputs=nl2, units=1, trainable=False)
        return states_logits, Qlogits

In [20]:
def model_loss(state_size, action_size, hidden_size, gamma,
               states, actions, next_states, rewards, dones, rates):
    ################################################ a = act(s)
    actions_logits = Act(states=states, hidden_size=hidden_size, action_size=action_size)
    actions_labels = tf.one_hot(indices=actions, depth=action_size, dtype=actions_logits.dtype)
    aloss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=actions_logits, 
                                                                      labels=actions_labels))
    ################################################ s', r = env(s, a)
    next_states_logits, Qs_logits = Env(actions=actions_labels, states=states, hidden_size=hidden_size, 
                                        action_size=action_size, state_size=state_size)
    next_states_labels = tf.nn.sigmoid(next_states)
    eloss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=next_states_logits,
                                                                   labels=next_states_labels))
    #################################################### s'', Q' = env(s', a')
    next_actions_logits = Act(states=next_states, hidden_size=hidden_size, action_size=action_size, reuse=True)
    next_states_logits, Qs2_logits = Env(actions=next_actions_logits, states=next_states, hidden_size=hidden_size, 
                                         action_size=action_size, state_size=state_size, reuse=True)
    #################################################### s'', Q' = env(~s', ~a')
    next_actions_logits = Act(states=next_states_logits, hidden_size=hidden_size, action_size=action_size, 
                              reuse=True)
    next_states_logits_, Qs2_logits_ = Env(actions=next_actions_logits, states=next_states_logits, 
                                           hidden_size=hidden_size, action_size=action_size, 
                                           state_size=state_size, reuse=True)
    ##################################################### GAN
    Qs = tf.reshape(Qs_logits, shape=[-1])
    Qs2 = tf.reshape(Qs2_logits, shape=[-1]) * (1-dones)
    Qs2_ = tf.reshape(Qs2_logits_, shape=[-1]) * (1-dones)
    eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=Qs, # GAN
                                                                    labels=rates)) # 0-1 real
    eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=(Qs2+Qs2_)/2, # GAN
                                                                    labels=tf.zeros_like(rates))) # min
    aloss2 = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=(Qs2+Qs2_)/2, # GAN
                                                                    labels=tf.ones_like(rates))) # max
    ########################################################## Qlearning
    targetQs = rewards + (Qs2+Qs2_)/2
    eloss += tf.reduce_mean(tf.square(Qs - targetQs))
    ######################################################### Curosity=intrinsic reward

    return actions_logits, aloss, eloss, aloss2

In [21]:
def model_opt(a_loss, e_loss, a_loss2, a_learning_rate, e_learning_rate):
    # Get weights and bias to update
    t_vars = tf.trainable_variables()
    a_vars = [var for var in t_vars if var.name.startswith('Act')]
    e_vars = [var for var in t_vars if var.name.startswith('Env')]

    # Optimize
    with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)): # Required for batchnorm (BN)
        a_opt = tf.train.AdamOptimizer(a_learning_rate).minimize(a_loss, var_list=a_vars)
        e_opt = tf.train.AdamOptimizer(e_learning_rate).minimize(e_loss, var_list=e_vars)
        a_opt2 = tf.train.AdamOptimizer(a_learning_rate).minimize(a_loss2, var_list=a_vars)
    return a_opt, e_opt, a_opt2

In [22]:
class Model:
    def __init__(self, state_size, action_size, hidden_size, a_learning_rate, e_learning_rate, gamma):

        # Data of the Model: make the data available inside the framework
        self.states, self.actions, self.next_states, self.rewards, self.dones, self.rates = model_input(
            state_size=state_size)

        # Create the Model: calculating the loss and forwad pass
        self.actions_logits, self.a_loss, self.e_loss, self.a_loss2 = model_loss(
            state_size=state_size, action_size=action_size, hidden_size=hidden_size, gamma=gamma, # model init
            states=self.states, actions=self.actions, next_states=self.next_states, 
            rewards=self.rewards, dones=self.dones, rates=self.rates) # model input
        
        # Update the model: backward pass and backprop
        self.a_opt, self.e_opt, self.a_opt2 = model_opt(a_loss=self.a_loss, 
                                                        e_loss=self.e_loss,
                                                        a_loss2=self.a_loss2, 
                                                        a_learning_rate=a_learning_rate,
                                                        e_learning_rate=e_learning_rate)

In [23]:
from collections import deque
class Memory():
    def __init__(self, max_size = 1000):
        self.buffer = deque(maxlen=max_size) # data batch
#     def sample(self, batch_size):
#         idx = np.random.choice(np.arange(len(self.buffer)), size=batch_size, replace=False)
#         return [self.buffer[ii] for ii in idx]

## Hyperparameters

One of the more difficult aspects of reinforcememt learning are the large number of hyperparameters. Not only are we tuning the network, but we're tuning the simulation.

In [24]:
env.observation_space, env.action_space

(Box(4,), Discrete(2))

In [25]:
# Exploration parameters
explore_start = 1.0            # exploration probability at start
explore_stop = 0.01           # minimum exploration probability 
decay_rate = 0.0001            # exponential decay rate for exploration prob

# Network parameters
state_size = 4
action_size = 2
hidden_size = 4*2             # number of units in each Q-network hidden layer
a_learning_rate = 1e-4         # Q-network learning rate
e_learning_rate = 1e-4         # Q-network learning rate

# Memory parameters
memory_size = int(1e5)            # memory capacity
batch_size = int(1e3)             # experience mini-batch size
gamma=0.99

In [26]:
# Reset/init the graph/session
graph = tf.reset_default_graph()

# Init the model
model = Model(action_size=action_size, state_size=state_size, hidden_size=hidden_size, gamma=gamma,
              a_learning_rate=a_learning_rate, e_learning_rate=e_learning_rate)

# Init the memory
memory = Memory(max_size=memory_size)

In [27]:
state = env.reset()
total_reward = 0
num_step = 0
for _ in range(memory_size):
    action = env.action_space.sample()
    next_state, reward, done, _ = env.step(action)
    rate = -1
    memory.buffer.append([state, action, next_state, reward, float(done), rate])
    num_step += 1 # memory incremented
    total_reward += reward
    state = next_state
    if done is True:
        rate = total_reward/500
        for idx in range(num_step): # episode length
            if memory.buffer[-1-idx][-1] == -1:
                memory.buffer[-1-idx][-1] = rate
        state = env.reset()
        total_reward = 0 # reset
        num_step = 0 # reset

## Training the model

Below we'll train our agent. If you want to watch it train, uncomment the `env.render()` line. This is slow because it's rendering the frames slower than the network can train. But, it's cool to watch the agent get better at the game.

In [ ]:
# Save/load the model and save for plotting
saver = tf.train.Saver()
episode_rewards_list, rewards_list = [], []
aloss_list, eloss_list, aloss2_list = [], [], []

# TF session for training
with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    #saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    total_step = 0 # Explore or exploit parameter
    episode_reward = deque(maxlen=100) # 100 episodes for running average/running mean/window

    # Training episodes/epochs
    for ep in range(11111):
        aloss_batch, eloss_batch, aloss2_batch = [], [], []
        total_reward = 0
        state = env.reset()
        num_step = 0
        rate = -1

        # Training steps/batches
        while True:
            # Explore (env) or Exploit (model)
            total_step += 1
            explore_p = explore_stop + (explore_start - explore_stop) * np.exp(-decay_rate * total_step) 
            if explore_p > np.random.rand():
                action = env.action_space.sample()
            else:
                action_logits = sess.run(model.actions_logits, feed_dict={model.states: state.reshape([1, -1])})
                action = np.argmax(action_logits)
            next_state, reward, done, _ = env.step(action)
            memory.buffer.append([state, action, next_state, reward, float(done), rate])
            num_step += 1 # momory added
            total_reward += reward
            state = next_state
            
            # Training with the maxrated minibatch
            batch = memory.buffer
            #for idx in range(memory_size// batch_size):
            while True:
                idx = np.random.choice(np.arange(memory_size// batch_size))
                states = np.array([each[0] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                actions = np.array([each[1] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                next_states = np.array([each[2] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                rewards = np.array([each[3] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                dones = np.array([each[4] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                rates = np.array([each[5] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                states = states[rates >= np.max(rates)]
                actions = actions[rates >= np.max(rates)]
                next_states = next_states[rates >= np.max(rates)]
                rewards = rewards[rates >= np.max(rates)]
                dones = dones[rates >= np.max(rates)]
                rates = rates[rates >= np.max(rates)]
                if np.count_nonzero(dones) > 0 and len(dones) > 1 and np.max(rates) > 0:
                    break
            aloss, _ = sess.run([model.a_loss, model.a_opt],
                                  feed_dict = {model.states: states, 
                                               model.actions: actions,
                                               model.next_states: next_states,
                                               model.rewards: rewards,
                                               model.dones: dones,
                                               model.rates: rates})
            eloss, _ = sess.run([model.e_loss, model.e_opt],
                                  feed_dict = {model.states: states, 
                                               model.actions: actions,
                                               model.next_states: next_states,
                                               model.rewards: rewards,
                                               model.dones: dones,
                                               model.rates: rates})
            aloss2, _= sess.run([model.a_loss2, model.a_opt2], 
                                 feed_dict = {model.states: states, 
                                              model.actions: actions,
                                              model.next_states: next_states,
                                              model.rewards: rewards,
                                              model.dones: dones,
                                              model.rates: rates})
            # print(len(dones), np.count_nonzero(dones), np.max(rates))
            aloss_batch.append(aloss)
            eloss_batch.append(eloss)
            aloss2_batch.append(aloss2)
            if done is True:
                break
                
        # Rating the latest played episode
        rate = total_reward/500 # update rate at the end/ when episode is done
        for idx in range(num_step): # episode length
            if memory.buffer[-1-idx][-1] == -1: # double-check the landmark/marked indexes
                memory.buffer[-1-idx][-1] = rate # rate the trajectory/data

        # Print out
        episode_reward.append(total_reward)
        print('Episode:{}'.format(ep),
              'meanR:{:.4f}'.format(np.mean(episode_reward)),
              'R:{:.4f}'.format(total_reward),
              'rate:{:.4f}'.format(rate),
              'aloss:{:.4f}'.format(np.mean(aloss_batch)),
              'eloss:{:.4f}'.format(np.mean(eloss_batch)),
              'aloss2:{:.4f}'.format(np.mean(aloss2_batch)),
              'exploreP:{:.4f}'.format(explore_p))

        # Ploting out
        episode_rewards_list.append([ep, np.mean(episode_reward)])
        rewards_list.append([ep, total_reward])
        aloss_list.append([ep, np.mean(aloss_batch)])
        eloss_list.append([ep, np.mean(eloss_batch)])
        aloss2_list.append([ep, np.mean(aloss2_batch)])
        
        # Break episode/epoch loop
        ## Option 1: Solve the First Version
        #The task is episodic, and in order to solve the environment, 
        #your agent must get an average score of +30 over 100 consecutive episodes.
        if np.mean(episode_reward) >= 500:
            break
            
    # At the end of all training episodes/epochs
    saver.save(sess, 'checkpoints/model.ckpt')

Episode:0 meanR:18.0000 R:18.0000 rate:0.0360 aloss:0.7235 eloss:2.9366 aloss2:0.6947 exploreP:0.9982
Episode:1 meanR:20.0000 R:22.0000 rate:0.0440 aloss:0.7214 eloss:2.9342 aloss2:0.6949 exploreP:0.9960
Episode:2 meanR:21.6667 R:25.0000 rate:0.0500 aloss:0.7013 eloss:2.9370 aloss2:0.6933 exploreP:0.9936
Episode:3 meanR:22.0000 R:23.0000 rate:0.0460 aloss:0.7317 eloss:2.9346 aloss2:0.6944 exploreP:0.9913
Episode:4 meanR:22.8000 R:26.0000 rate:0.0520 aloss:0.6983 eloss:2.9423 aloss2:0.6913 exploreP:0.9888
Episode:5 meanR:21.6667 R:16.0000 rate:0.0320 aloss:0.6992 eloss:2.9412 aloss2:0.6914 exploreP:0.9872
Episode:6 meanR:20.5714 R:14.0000 rate:0.0280 aloss:0.7014 eloss:2.9487 aloss2:0.6905 exploreP:0.9858
Episode:7 meanR:19.6250 R:13.0000 rate:0.0260 aloss:0.6940 eloss:2.9467 aloss2:0.6902 exploreP:0.9846
Episode:8 meanR:22.8889 R:49.0000 rate:0.0980 aloss:0.6970 eloss:2.9337 aloss2:0.6910 exploreP:0.9798
Episode:9 meanR:23.6000 R:30.0000 rate:0.0600 aloss:0.6980 eloss:2.9168 aloss2:0.6

Episode:80 meanR:22.8025 R:20.0000 rate:0.0400 aloss:0.7334 eloss:2.6894 aloss2:0.8095 exploreP:0.8330
Episode:81 meanR:22.9024 R:31.0000 rate:0.0620 aloss:0.7404 eloss:2.6632 aloss2:0.8164 exploreP:0.8305
Episode:82 meanR:22.8434 R:18.0000 rate:0.0360 aloss:0.7392 eloss:2.6424 aloss2:0.8245 exploreP:0.8290
Episode:83 meanR:22.7143 R:12.0000 rate:0.0240 aloss:0.7397 eloss:2.6401 aloss2:0.8267 exploreP:0.8280
Episode:84 meanR:22.6000 R:13.0000 rate:0.0260 aloss:0.7761 eloss:2.6322 aloss2:0.8276 exploreP:0.8270
Episode:85 meanR:22.4535 R:10.0000 rate:0.0200 aloss:0.7355 eloss:2.6265 aloss2:0.8308 exploreP:0.8262
Episode:86 meanR:22.7011 R:44.0000 rate:0.0880 aloss:0.7466 eloss:2.6074 aloss2:0.8393 exploreP:0.8226
Episode:87 meanR:22.5568 R:10.0000 rate:0.0200 aloss:0.7595 eloss:2.5907 aloss2:0.8442 exploreP:0.8218
Episode:88 meanR:22.6854 R:34.0000 rate:0.0680 aloss:0.7447 eloss:2.5727 aloss2:0.8489 exploreP:0.8190
Episode:89 meanR:22.8222 R:35.0000 rate:0.0700 aloss:0.7448 eloss:2.5480 

Episode:159 meanR:34.7800 R:13.0000 rate:0.0260 aloss:0.8176 eloss:2.3711 aloss2:1.0916 exploreP:0.6227
Episode:160 meanR:35.0200 R:41.0000 rate:0.0820 aloss:0.8705 eloss:2.3721 aloss2:1.0989 exploreP:0.6202
Episode:161 meanR:35.1100 R:25.0000 rate:0.0500 aloss:0.8142 eloss:2.3457 aloss2:1.1298 exploreP:0.6187
Episode:162 meanR:35.2300 R:32.0000 rate:0.0640 aloss:0.8050 eloss:2.3252 aloss2:1.1294 exploreP:0.6167
Episode:163 meanR:35.5600 R:55.0000 rate:0.1100 aloss:0.7980 eloss:2.3109 aloss2:1.1420 exploreP:0.6134
Episode:164 meanR:35.4100 R:18.0000 rate:0.0360 aloss:0.8337 eloss:2.2790 aloss2:1.1449 exploreP:0.6123
Episode:165 meanR:35.5800 R:35.0000 rate:0.0700 aloss:0.7879 eloss:2.2567 aloss2:1.1678 exploreP:0.6102
Episode:166 meanR:35.5500 R:18.0000 rate:0.0360 aloss:0.8281 eloss:2.2637 aloss2:1.1705 exploreP:0.6091
Episode:167 meanR:35.4700 R:21.0000 rate:0.0420 aloss:0.8186 eloss:2.2488 aloss2:1.1682 exploreP:0.6079
Episode:168 meanR:35.3200 R:12.0000 rate:0.0240 aloss:0.7955 elo

Episode:238 meanR:31.1700 R:28.0000 rate:0.0560 aloss:0.7945 eloss:1.8555 aloss2:1.5351 exploreP:0.5032
Episode:239 meanR:31.0700 R:17.0000 rate:0.0340 aloss:0.7522 eloss:1.8235 aloss2:1.5706 exploreP:0.5024
Episode:240 meanR:29.8300 R:13.0000 rate:0.0260 aloss:0.7741 eloss:1.8413 aloss2:1.5812 exploreP:0.5017
Episode:241 meanR:29.5100 R:18.0000 rate:0.0360 aloss:0.7899 eloss:1.8375 aloss2:1.5439 exploreP:0.5008
Episode:242 meanR:29.3500 R:18.0000 rate:0.0360 aloss:0.7750 eloss:1.8213 aloss2:1.5650 exploreP:0.5000
Episode:243 meanR:28.7700 R:17.0000 rate:0.0340 aloss:0.7799 eloss:1.7983 aloss2:1.5798 exploreP:0.4991
Episode:244 meanR:28.6900 R:13.0000 rate:0.0260 aloss:0.7918 eloss:1.7794 aloss2:1.5610 exploreP:0.4985
Episode:245 meanR:28.7600 R:19.0000 rate:0.0380 aloss:0.7627 eloss:1.7928 aloss2:1.5937 exploreP:0.4976
Episode:246 meanR:28.6300 R:18.0000 rate:0.0360 aloss:0.7937 eloss:1.8025 aloss2:1.5924 exploreP:0.4967
Episode:247 meanR:27.3100 R:15.0000 rate:0.0300 aloss:0.7682 elo

Episode:317 meanR:16.2400 R:19.0000 rate:0.0380 aloss:0.7023 eloss:1.7217 aloss2:1.7502 exploreP:0.4510
Episode:318 meanR:16.0900 R:14.0000 rate:0.0280 aloss:0.7069 eloss:1.7240 aloss2:1.7245 exploreP:0.4504
Episode:319 meanR:16.2800 R:40.0000 rate:0.0800 aloss:0.7061 eloss:1.7248 aloss2:1.7654 exploreP:0.4487
Episode:320 meanR:16.1700 R:8.0000 rate:0.0160 aloss:0.6897 eloss:1.7475 aloss2:1.8579 exploreP:0.4483
Episode:321 meanR:16.1100 R:12.0000 rate:0.0240 aloss:0.6991 eloss:1.6606 aloss2:1.7639 exploreP:0.4478
Episode:322 meanR:16.0000 R:12.0000 rate:0.0240 aloss:0.7086 eloss:1.6983 aloss2:1.7274 exploreP:0.4472
Episode:323 meanR:15.8800 R:13.0000 rate:0.0260 aloss:0.7066 eloss:1.7737 aloss2:1.8349 exploreP:0.4467
Episode:324 meanR:15.8900 R:19.0000 rate:0.0380 aloss:0.7025 eloss:1.7784 aloss2:1.7347 exploreP:0.4459
Episode:325 meanR:15.4700 R:10.0000 rate:0.0200 aloss:0.7021 eloss:1.6817 aloss2:1.7917 exploreP:0.4454
Episode:326 meanR:15.4000 R:11.0000 rate:0.0220 aloss:0.7017 elos

...

Episode:949 meanR:40.1300 R:28.0000 rate:0.0560 aloss:0.7075 eloss:1.3866 aloss2:2.7569 exploreP:0.0480
Episode:950 meanR:39.9100 R:42.0000 rate:0.0840 aloss:0.7114 eloss:1.2997 aloss2:2.7762 exploreP:0.0478
Episode:951 meanR:39.9500 R:52.0000 rate:0.1040 aloss:0.7043 eloss:1.3423 aloss2:2.7733 exploreP:0.0476
Episode:952 meanR:39.9200 R:24.0000 rate:0.0480 aloss:0.7081 eloss:1.3255 aloss2:2.7933 exploreP:0.0475
Episode:953 meanR:39.6300 R:40.0000 rate:0.0800 aloss:0.7193 eloss:1.2887 aloss2:2.7912 exploreP:0.0474
Episode:954 meanR:39.6800 R:40.0000 rate:0.0800 aloss:0.6939 eloss:1.3122 aloss2:2.7984 exploreP:0.0472
Episode:955 meanR:39.6700 R:35.0000 rate:0.0700 aloss:0.6806 eloss:1.3600 aloss2:2.7711 exploreP:0.0471
Episode:956 meanR:39.6700 R:43.0000 rate:0.0860 aloss:0.6947 eloss:1.3333 aloss2:2.7273 exploreP:0.0469
Episode:957 meanR:39.7400 R:31.0000 rate:0.0620 aloss:0.7071 eloss:1.3101 aloss2:2.7801 exploreP:0.0468
Episode:958 meanR:39.5300 R:36.0000 rate:0.0720 aloss:0.7052 elo

Episode:1028 meanR:41.9500 R:83.0000 rate:0.1660 aloss:0.7103 eloss:1.3274 aloss2:2.7507 exploreP:0.0370
Episode:1029 meanR:41.4300 R:35.0000 rate:0.0700 aloss:0.6903 eloss:1.3240 aloss2:2.7594 exploreP:0.0369
Episode:1030 meanR:41.6200 R:75.0000 rate:0.1500 aloss:0.7194 eloss:1.3096 aloss2:2.7563 exploreP:0.0367
Episode:1031 meanR:41.8400 R:76.0000 rate:0.1520 aloss:0.7261 eloss:1.3564 aloss2:2.7825 exploreP:0.0365
Episode:1032 meanR:41.9400 R:48.0000 rate:0.0960 aloss:0.6883 eloss:1.3358 aloss2:2.7640 exploreP:0.0363
Episode:1033 meanR:42.5200 R:86.0000 rate:0.1720 aloss:0.7185 eloss:1.3329 aloss2:2.7601 exploreP:0.0361
Episode:1034 meanR:42.2800 R:22.0000 rate:0.0440 aloss:0.7242 eloss:1.2837 aloss2:2.7754 exploreP:0.0361
Episode:1035 meanR:42.2800 R:31.0000 rate:0.0620 aloss:0.6951 eloss:1.5177 aloss2:2.7414 exploreP:0.0360
Episode:1036 meanR:42.4800 R:65.0000 rate:0.1300 aloss:0.7221 eloss:1.3358 aloss2:2.7185 exploreP:0.0358
Episode:1037 meanR:42.7000 R:60.0000 rate:0.1200 aloss:

Episode:1107 meanR:38.6600 R:16.0000 rate:0.0320 aloss:0.7007 eloss:1.3569 aloss2:2.6747 exploreP:0.0304
Episode:1108 meanR:38.5700 R:24.0000 rate:0.0480 aloss:0.7274 eloss:1.3368 aloss2:2.7409 exploreP:0.0304
Episode:1109 meanR:38.6200 R:34.0000 rate:0.0680 aloss:0.7279 eloss:1.3041 aloss2:2.7654 exploreP:0.0303
Episode:1110 meanR:38.1800 R:32.0000 rate:0.0640 aloss:0.7530 eloss:1.2912 aloss2:2.7835 exploreP:0.0302
Episode:1111 meanR:38.3600 R:47.0000 rate:0.0940 aloss:0.7452 eloss:1.2898 aloss2:2.7669 exploreP:0.0302
Episode:1112 meanR:38.3700 R:26.0000 rate:0.0520 aloss:0.7315 eloss:1.3146 aloss2:2.7554 exploreP:0.0301
Episode:1113 meanR:37.9000 R:29.0000 rate:0.0580 aloss:0.7362 eloss:1.2863 aloss2:2.8248 exploreP:0.0300
Episode:1114 meanR:37.7100 R:23.0000 rate:0.0460 aloss:0.7075 eloss:1.3160 aloss2:2.8049 exploreP:0.0300
Episode:1115 meanR:37.5600 R:23.0000 rate:0.0460 aloss:0.7110 eloss:1.2981 aloss2:2.7274 exploreP:0.0299
Episode:1116 meanR:36.8400 R:35.0000 rate:0.0700 aloss:

Episode:1186 meanR:27.2200 R:33.0000 rate:0.0660 aloss:0.7174 eloss:1.3782 aloss2:2.7633 exploreP:0.0266
Episode:1187 meanR:27.1300 R:27.0000 rate:0.0540 aloss:0.7497 eloss:1.3204 aloss2:2.7823 exploreP:0.0265
Episode:1188 meanR:27.1700 R:22.0000 rate:0.0440 aloss:0.7412 eloss:1.3102 aloss2:2.7684 exploreP:0.0265
Episode:1189 meanR:27.2500 R:28.0000 rate:0.0560 aloss:0.7078 eloss:1.3549 aloss2:2.6819 exploreP:0.0265
Episode:1190 meanR:27.0900 R:28.0000 rate:0.0560 aloss:0.7261 eloss:1.3084 aloss2:2.7222 exploreP:0.0264
Episode:1191 meanR:26.9800 R:36.0000 rate:0.0720 aloss:0.7370 eloss:1.3268 aloss2:2.7503 exploreP:0.0264
Episode:1192 meanR:26.8400 R:28.0000 rate:0.0560 aloss:0.7604 eloss:1.2722 aloss2:2.8028 exploreP:0.0263
Episode:1193 meanR:26.8100 R:38.0000 rate:0.0760 aloss:0.7169 eloss:1.3384 aloss2:2.7768 exploreP:0.0262
Episode:1194 meanR:26.6000 R:27.0000 rate:0.0540 aloss:0.7086 eloss:1.3377 aloss2:2.7897 exploreP:0.0262
Episode:1195 meanR:26.7600 R:42.0000 rate:0.0840 aloss:

Episode:1265 meanR:27.3300 R:34.0000 rate:0.0680 aloss:0.7131 eloss:1.3270 aloss2:2.7639 exploreP:0.0233
Episode:1266 meanR:27.5000 R:34.0000 rate:0.0680 aloss:0.7657 eloss:1.3247 aloss2:2.7198 exploreP:0.0233
Episode:1267 meanR:27.6600 R:32.0000 rate:0.0640 aloss:0.7581 eloss:1.3140 aloss2:2.7421 exploreP:0.0233
Episode:1268 meanR:27.5800 R:25.0000 rate:0.0500 aloss:0.6860 eloss:1.3230 aloss2:2.6946 exploreP:0.0232
Episode:1269 meanR:27.5300 R:16.0000 rate:0.0320 aloss:0.7273 eloss:1.3039 aloss2:2.7334 exploreP:0.0232
Episode:1270 meanR:27.3800 R:16.0000 rate:0.0320 aloss:0.7320 eloss:1.3690 aloss2:2.6744 exploreP:0.0232
Episode:1271 meanR:27.3200 R:25.0000 rate:0.0500 aloss:0.7136 eloss:1.3613 aloss2:2.7019 exploreP:0.0231
Episode:1272 meanR:27.2900 R:16.0000 rate:0.0320 aloss:0.7405 eloss:1.3148 aloss2:2.7949 exploreP:0.0231
Episode:1273 meanR:27.4300 R:36.0000 rate:0.0720 aloss:0.7191 eloss:1.3115 aloss2:2.7362 exploreP:0.0231
Episode:1274 meanR:27.3800 R:32.0000 rate:0.0640 aloss:

# Visualizing training

Below I'll plot the total rewards for each episode. I'm plotting the rolling average too, in blue.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

def running_mean(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / N 

In [ ]:
eps, arr = np.array(episode_rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(aloss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Act losses')

In [ ]:
eps, arr = np.array(eloss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Env losses')

In [ ]:
eps, arr = np.array(aloss2_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Act losses 2')

## Testing

Let's checkout how our trained agent plays the game.

In [21]:
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    
    # Episodes/epochs
    for _ in range(10):
        state = env.reset()
        total_reward = 0

        # Steps/batches
        while True:
            env.render()
            action_logits = sess.run(model.actions_logits, feed_dict={model.states: state.reshape([1, -1])})
            action = np.argmax(action_logits)
            state, reward, done, _ = env.step(action)
            total_reward += reward
            if done:
                print('total_reward: {}'.format(total_reward))
                break
                
env.close()

INFO:tensorflow:Restoring parameters from checkpoints/model.ckpt
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
